<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Day%20%232-Intelligent-Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Date: 6th January 2020

# Source: Taken verbatim from DataHack Summit 2019 
# HackSession - Enabling Intelligent Search using Q&A models - Abhishek Jha, Priya Shree & Atul Singh (PhD) 

#!pip install deeppavlov

In [0]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
!ls /root/nltk_data/corpora/gutenberg/

austen-emma.txt		 carroll-alice.txt	  README
austen-persuasion.txt	 chesterton-ball.txt	  shakespeare-caesar.txt
austen-sense.txt	 chesterton-brown.txt	  shakespeare-hamlet.txt
bible-kjv.txt		 chesterton-thursday.txt  shakespeare-macbeth.txt
blake-poems.txt		 edgeworth-parents.txt	  whitman-leaves.txt
bryant-stories.txt	 melville-moby_dick.txt
burgess-busterbrown.txt  milton-paradise.txt


In [0]:
from nltk.corpus import gutenberg
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.core.display import display, HTML

In [0]:
import torch
from deeppavlov import build_model, configs

In [0]:
book_name = 'carroll-alice.txt'

In [0]:
display(HTML('<h2> No. of paragraphs: ' +str(len(gutenberg.paras(book_name))) + ' </h2> '))

In [0]:
display(HTML('<h2> No. of words: ' +str(len(gutenberg.words(book_name))) + ' </h2> '))

In [0]:
gutenberg.paras(book_name)[4]

In [0]:
para_list =[]

for para in gutenberg.paras(book_name):
  sentence_list = []

  for sentence in para:
    sentence_list.append(TreebankWordDetokenizer().detokenize(sentence))

  para_list.append(" ".join((sent for sent in sentence_list))) 

In [0]:
para_list[4]

"There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself ,' Oh dear! Oh dear! I shall be late!' ( when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural ); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT - POCKET, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat - pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit - hole under the hedge."

In [0]:
?TreebankWordDetokenizer().detokenize

In [0]:
count_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))

In [0]:
count_vec_total_text = count_vectorizer.fit_transform(para_list)

In [0]:
#
question = "What did the rabbit take out of his waistcoat pocket"

In [0]:
count_vec_question = count_vectorizer.transform([question])

In [0]:
print(np.shape(count_vec_total_text))

(817, 10658)


In [0]:
print(np.shape(count_vec_question))

(1, 10658)


In [0]:
distance_array = cosine_similarity(count_vec_question, count_vec_total_text)

In [0]:
np.shape(distance_array)

(1, 817)

In [0]:
distance_array = distance_array[0] 

In [0]:
distance_array

array([0.        , 0.18257419, 0.        , 0.06085806, 0.42802583,
       0.        , 0.07106691, 0.0402259 , 0.        , 0.        ,
       0.        , 0.0255655 , 0.0789337 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.09774528, 0.        , 0.02879561, 0.        , 0.        ,
       0.        , 0.04778185, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03178209, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.05227084, 0.        , 0.        ,
       0.10540926, 0.        , 0.07332356, 0.        , 0.        ,
       0.08908708, 0.05832118, 0.        , 0.        , 0.     

In [0]:
top_n = 5

In [0]:
top_passages = []

In [0]:
distance_array.argsort()[::-1][:top_n]

array([  4, 456,   1, 712, 574])

In [0]:
for idx in  [4, 456, 1, 712, 574]:
  print(" {:03.2f}".format(distance_array[idx]))
 

 0.43
 0.27
 0.18
 0.18
 0.18


In [0]:
for idx in distance_array.argsort()[::-1][:top_n]:
  print(idx, ' || ', para_list[idx] )
  print("="*20)
  top_passages.append(para_list[idx])

4  ||  There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself ,' Oh dear! Oh dear! I shall be late!' ( when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural ); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT - POCKET, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat - pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit - hole under the hedge.
456  ||  ' Did you say " What a pity!"?' the Rabbit asked.
1  ||  CHAPTER I. Down the Rabbit - Hole
712  ||  ' You did!' said the Hatter.
574  ||  ' So he did, so he did ,' said the Gryphon, sighing in his turn; and both creatures hid their faces in 

In [0]:
#!python -m deeppavlov install squad_bert

In [0]:
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad_bert, download = True)

2020-01-06 10:47:25.703 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/cased_L-12_H-768_A-12.zip download because of matching hashes
2020-01-06 10:47:26.988 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_bert.tar.gz download because of matching hashes


Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-01-06 10:47:52.686 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_bert/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_bert/model


In [0]:
predicted_answer_list = []

for passage in top_passages:
  predicted_answer = model([passage], [question])
  predicted_answer_list.append(predicted_answer)
  print(predicted_answer)

[['a watch'], [562], [135285.109375]]
[['the Rabbit asked.'], [32], [0.6844851970672607]]
[['CHAPTER I. Down the Rabbit - Hole'], [0], [0.4659876227378845]]
[['You did'], [2], [4.523017406463623]]
[['both creatures hid their faces in their paws'], [69], [761.21533203125]]


In [0]:
!pip install transformers

     |████████████████████████████████| 450kB 5.0MB/s 
     |████████████████████████████████| 1.0MB 38.4MB/s 
     |████████████████████████████████| 870kB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=ec7ca69f24f002b2db664791076920a74942e5f03e73c304a92d0eafe7daf6b0
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import BertForQuestionAnswering, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [0]:
input_text = " [CLS] " + question + " [SEP] " + top_passages[0]

In [0]:
tokenized_text = tokenizer.tokenize(input_text)

input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

In [0]:
print(top_passages[0])

There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself ,' Oh dear! Oh dear! I shall be late!' ( when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural ); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT - POCKET, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat - pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit - hole under the hedge.


In [0]:
print(tokenized_text)

['[CLS]', 'what', 'did', 'the', 'rabbit', 'take', 'out', 'of', 'his', 'waist', '##coat', 'pocket', '[SEP]', 'there', 'was', 'nothing', 'so', 'very', 'remarkable', 'in', 'that', ';', 'nor', 'did', 'alice', 'think', 'it', 'so', 'very', 'much', 'out', 'of', 'the', 'way', 'to', 'hear', 'the', 'rabbit', 'say', 'to', 'itself', ',', "'", 'oh', 'dear', '!', 'oh', 'dear', '!', 'i', 'shall', 'be', 'late', '!', "'", '(', 'when', 'she', 'thought', 'it', 'over', 'afterwards', ',', 'it', 'occurred', 'to', 'her', 'that', 'she', 'ought', 'to', 'have', 'wondered', 'at', 'this', ',', 'but', 'at', 'the', 'time', 'it', 'all', 'seemed', 'quite', 'natural', ')', ';', 'but', 'when', 'the', 'rabbit', 'actually', 'took', 'a', 'watch', 'out', 'of', 'its', 'waist', '##coat', '-', 'pocket', ',', 'and', 'looked', 'at', 'it', ',', 'and', 'then', 'hurried', 'on', ',', 'alice', 'started', 'to', 'her', 'feet', ',', 'for', 'it', 'flashed', 'across', 'her', 'mind', 'that', 'she', 'had', 'never', 'before', 'seen', 'a', '

In [0]:
for tup in zip(tokenized_text, input_ids):
  print(tup)

('[CLS]', 101)
('what', 2054)
('did', 2106)
('the', 1996)
('rabbit', 10442)
('take', 2202)
('out', 2041)
('of', 1997)
('his', 2010)
('waist', 5808)
('##coat', 16531)
('pocket', 4979)
('[SEP]', 102)
('there', 2045)
('was', 2001)
('nothing', 2498)
('so', 2061)
('very', 2200)
('remarkable', 9487)
('in', 1999)
('that', 2008)
(';', 1025)
('nor', 4496)
('did', 2106)
('alice', 5650)
('think', 2228)
('it', 2009)
('so', 2061)
('very', 2200)
('much', 2172)
('out', 2041)
('of', 1997)
('the', 1996)
('way', 2126)
('to', 2000)
('hear', 2963)
('the', 1996)
('rabbit', 10442)
('say', 2360)
('to', 2000)
('itself', 2993)
(',', 1010)
("'", 1005)
('oh', 2821)
('dear', 6203)
('!', 999)
('oh', 2821)
('dear', 6203)
('!', 999)
('i', 1045)
('shall', 4618)
('be', 2022)
('late', 2397)
('!', 999)
("'", 1005)
('(', 1006)
('when', 2043)
('she', 2016)
('thought', 2245)
('it', 2009)
('over', 2058)
('afterwards', 5728)
(',', 1010)
('it', 2009)
('occurred', 4158)
('to', 2000)
('her', 2014)
('that', 2008)
('she', 2016)
(

In [0]:
# identify the question & the answer paragraph
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

In [0]:
start_score, end_score = model(torch.tensor([input_ids]), token_type_ids= torch.tensor([token_type_ids]))

In [0]:
len(start_score[0]), len(input_ids)

(183, 183)

In [0]:
start_score[0]

tensor([-5.1969, -7.5432, -8.4612, -7.9589, -8.0555, -7.3838, -7.9336, -7.9534,
        -7.6459, -8.7929, -9.5364, -9.9329, -5.1969, -7.5316, -7.8964, -6.0435,
        -7.4493, -8.3221, -7.5872, -6.9114, -6.6398, -8.5586, -7.5670, -7.7474,
        -5.3059, -7.4327, -7.1054, -7.5455, -8.3294, -8.0227, -6.9036, -8.0763,
        -8.5485, -8.2907, -7.6083, -8.1075, -7.6163, -6.0191, -8.5698, -8.4949,
        -7.5469, -8.5402, -5.6716, -5.2807, -7.7538, -8.1537, -6.0293, -7.6878,
        -8.1158, -5.4844, -7.5109, -7.8663, -6.3497, -8.6834, -8.1787, -8.0117,
        -7.2577, -7.6060, -7.7316, -7.8688, -8.6228, -7.6330, -8.4917, -7.3532,
        -7.3277, -8.4594, -7.7542, -8.4035, -7.3356, -8.2420, -8.5340, -8.7207,
        -7.5331, -8.9373, -7.6748, -8.7764, -7.8257, -6.6288, -6.5740, -7.9892,
        -6.4077, -7.5728, -6.3276, -6.6751, -6.0519, -7.6977, -6.4876, -4.2391,
        -0.2670,  2.7878,  1.1234,  0.3546,  2.5449,  5.0744,  3.7283, -2.7159,
        -3.4897, -1.6514, -0.7780, -4.56

In [0]:
torch.argmax(start_score), torch.max(start_score)

(tensor(93), tensor(5.0744, grad_fn=<MaxBackward1>))

In [0]:
torch.argmax(end_score), torch.max(end_score)

(tensor(94), tensor(5.2814, grad_fn=<MaxBackward1>))

In [0]:
torch.argmax(torch.nn.Softmax()(start_score[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor(93)

In [0]:
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

In [0]:
print(all_tokens)

['[CLS]', 'what', 'did', 'the', 'rabbit', 'take', 'out', 'of', 'his', 'waist', '##coat', 'pocket', '[SEP]', 'there', 'was', 'nothing', 'so', 'very', 'remarkable', 'in', 'that', ';', 'nor', 'did', 'alice', 'think', 'it', 'so', 'very', 'much', 'out', 'of', 'the', 'way', 'to', 'hear', 'the', 'rabbit', 'say', 'to', 'itself', ',', "'", 'oh', 'dear', '!', 'oh', 'dear', '!', 'i', 'shall', 'be', 'late', '!', "'", '(', 'when', 'she', 'thought', 'it', 'over', 'afterwards', ',', 'it', 'occurred', 'to', 'her', 'that', 'she', 'ought', 'to', 'have', 'wondered', 'at', 'this', ',', 'but', 'at', 'the', 'time', 'it', 'all', 'seemed', 'quite', 'natural', ')', ';', 'but', 'when', 'the', 'rabbit', 'actually', 'took', 'a', 'watch', 'out', 'of', 'its', 'waist', '##coat', '-', 'pocket', ',', 'and', 'looked', 'at', 'it', ',', 'and', 'then', 'hurried', 'on', ',', 'alice', 'started', 'to', 'her', 'feet', ',', 'for', 'it', 'flashed', 'across', 'her', 'mind', 'that', 'she', 'had', 'never', 'before', 'seen', 'a', '

In [0]:
predicted_answer = " ".join(all_tokens[torch.argmax(start_score): torch.argmax(end_score)+ 1]) 
predicted_answer

'a watch'

'a'